In [1]:
%matplotlib inline
from dimod import BQM
from neal import SimulatedAnnealingSampler
import pandas as pd
import numpy as np

# Setup Data

First of all, we need to setup all the distances for each point in the graph.

![problem graph](./assets/graph.png)

In [2]:
idx = pd.Index(['A', 'B', 'C', 'D', 'E'])
df = pd.DataFrame([
        np.array([0, 5.8, 5.8, 7.6, 8.8], dtype=float),
        np.array([5.8, 0, 2.4, 4.3, 4.3], dtype=float),
        np.array([5.8, 2.4, 0, 3.5, 4.7], dtype=float),
        np.array([7.6, 4.3, 3.5, 0, 1.4], dtype=float),
        np.array([8.8, 4.3, 4.7, 1.4, 0], dtype=float),
    ], 
    columns=['A', 'B', 'C', 'D', 'E'], 
    index=idx)
df

,A,B,C,D,E
A,0.0,5.8,5.8,7.6,8.8
B,5.8,0.0,2.4,4.3,4.3
C,5.8,2.4,0.0,3.5,4.7
D,7.6,4.3,3.5,0.0,1.4
E,8.8,4.3,4.7,1.4,0.0


---

# Qubo Problem creation

Using the Dwave's Ocean SDK, we'll map these points to a QUBO problem. The idea, is to create a model wich each variable is correlated with a timestamp and a poistion.

$$ \sum^{4}_{t=1} \sum^{4}_{\substack{i=0\\ \text{if t $\neq$ 1}\\ \text{else i=1}}} \sum^{4}_{\substack{j=0 \\ \text{if t $\neq$ 1}\\ \text{else j=1} \\ \& \\ j \neq  i}} Q_{ij} p_{tij} + P(\sum_{t,i,j}p_{tij} - 4) + P(\sum^{4}_{i=0}\sum^{4}_{\substack{j=0 \\ j \neq  i}}(\sum^{4}_{t=1}p_{tij} - 1))$$

Each $p_{ti}$ is a binary variable.

In [32]:
steps = df.shape[0]
P = 10

linear_terms = {}

following_path_constraint = [[] for _ in range(steps)]
every_timestep_sum_up_to_one = [[] for _ in range(steps-1)]
only_one_movement_per_step = [[] for _ in range(steps)]


for t in range(1, steps):
    if(t == 1):
        row = df.iloc[0].tolist()    
        for next_point in range(1, steps):
            val = row[next_point]
            term = f'p_10{next_point}'
            linear_terms[term] = val
            every_timestep_sum_up_to_one[t-1].append((term, 1))
            only_one_movement_per_step[next_point].append((term, 1))
            following_path_constraint[next_point].append((term,1))
            
    else:
        for prev in range((1 if t==2 else 0), steps):
            row = df.iloc[prev].tolist()
            for next_point in range(steps):
                if(prev == next_point):
                    continue
                val = row[next_point]
                term = f'p_{t}{prev}{next_point}'
                linear_terms[term] = val   
                every_timestep_sum_up_to_one[t-1].append((term, 1))
                only_one_movement_per_step[next_point].append((term, 1))
                

linear_terms, every_timestep_sum_up_to_one, only_one_movement_per_step

({'p_101': 5.8,
  'p_102': 5.8,
  'p_103': 7.6,
  'p_104': 8.8,
  'p_210': 5.8,
  'p_212': 2.4,
  'p_213': 4.3,
  'p_214': 4.3,
  'p_220': 5.8,
  'p_221': 2.4,
  'p_223': 3.5,
  'p_224': 4.7,
  'p_230': 7.6,
  'p_231': 4.3,
  'p_232': 3.5,
  'p_234': 1.4,
  'p_240': 8.8,
  'p_241': 4.3,
  'p_242': 4.7,
  'p_243': 1.4,
  'p_301': 5.8,
  'p_302': 5.8,
  'p_303': 7.6,
  'p_304': 8.8,
  'p_310': 5.8,
  'p_312': 2.4,
  'p_313': 4.3,
  'p_314': 4.3,
  'p_320': 5.8,
  'p_321': 2.4,
  'p_323': 3.5,
  'p_324': 4.7,
  'p_330': 7.6,
  'p_331': 4.3,
  'p_332': 3.5,
  'p_334': 1.4,
  'p_340': 8.8,
  'p_341': 4.3,
  'p_342': 4.7,
  'p_343': 1.4,
  'p_401': 5.8,
  'p_402': 5.8,
  'p_403': 7.6,
  'p_404': 8.8,
  'p_410': 5.8,
  'p_412': 2.4,
  'p_413': 4.3,
  'p_414': 4.3,
  'p_420': 5.8,
  'p_421': 2.4,
  'p_423': 3.5,
  'p_424': 4.7,
  'p_430': 7.6,
  'p_431': 4.3,
  'p_432': 3.5,
  'p_434': 1.4,
  'p_440': 8.8,
  'p_441': 4.3,
  'p_442': 4.7,
  'p_443': 1.4},
 [[('p_101', 1), ('p_102', 1), ('p_103'

In [33]:
qubo = BQM(vartype="BINARY")
qubo.add_linear_from(linear_terms)

for constraint in every_timestep_sum_up_to_one:
    qubo.add_linear_equality_constraint(constraint, P, -1)
    
for constraint in only_one_movement_per_step:
    qubo.add_linear_equality_constraint(constraint, P, -1)

In [34]:
sampler = SimulatedAnnealingSampler()
sampleset = sampler.sample(qubo, num_reads=1000)
print(sampleset)

    p_101 p_102 p_103 p_104 p_210 p_212 p_213 p_214 ... p_443 energy num_oc.
0       1     0     0     0     0     0     0     0 ...     0   21.0       1
6       1     0     0     0     0     0     0     0 ...     1   21.0       1
8       1     0     0     0     0     0     0     0 ...     0   21.0       1
12      1     0     0     0     0     0     0     0 ...     1   21.0       1
20      0     1     0     0     0     0     0     0 ...     1   21.0       1
24      1     0     0     0     0     0     0     0 ...     0   21.0       1
35      0     1     0     0     0     0     0     0 ...     0   21.0       1
36      0     1     0     0     0     0     0     0 ...     0   21.0       1
41      0     1     0     0     0     0     0     0 ...     1   21.0       1
42      1     0     0     0     0     1     0     0 ...     1   21.0       1
44      0     1     0     0     0     0     0     0 ...     0   21.0       1
45      1     0     0     0     0     1     0     0 ...     1   21.0       1

In [35]:
result = sampleset.to_pandas_dataframe()
print('Saving CSV file...')
result.to_csv("final-result.csv")

Saving CSV file...


In [36]:
min_energy = result['energy'].min()
print('min energy SimulatedAnnealingSampler(): ', min_energy)

min energy SimulatedAnnealingSampler():  21.00000000000182


In [37]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

lowest_energies = sampleset.lowest().to_pandas_dataframe()
lowest_energies

,p_101,p_102,p_103,p_104,p_210,p_212,p_213,p_214,p_220,p_221,p_223,p_224,p_230,p_231,p_232,p_234,p_240,p_241,p_242,p_243,p_301,p_302,p_303,p_304,p_310,p_312,p_313,p_314,p_320,p_321,p_323,p_324,p_330,p_331,p_332,p_334,p_340,p_341,p_342,p_343,p_401,p_402,p_403,p_404,p_410,p_412,p_413,p_414,p_420,p_421,p_423,p_424,p_430,p_431,p_432,p_434,p_440,p_441,p_442,p_443,energy,num_occurrences
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,21.0,1
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,21.0,1
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,21.0,1
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,21.0,1
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,21.0,1
5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,21.0,1
6,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,21.0,1
7,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,21.0,1
8,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,21.0,1
9,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,21.0,1
